## First things first...

You can use Instructor for many things. Here, we will use it to extract structured info from unstructured text.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

## Now let's get started

In [ ]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [ ]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field
from typing import Optional, List
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')

inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class UserDetail(BaseModel):
    chain_of_thought: str = Field(description="Think step by step.")
    name: str = Field(description="Name of the user.")
    age: int = Field(description="Age of the user.")
    email: Optional[str] = Field(description="Email of the user.")

user = client.chat.completions.create(
    model="llama3-70b-8192" if inference_provider == "groq" else "gpt-4-turbo",
    response_model=UserDetail, # this is Instructor at work!
    temperature=0.0,
    messages=[{"role": "user", "content": "Jason is the user and he's 25 years older than Rick who was born 3 years ago."}]
    )

print(user.model_dump_json(indent=4))


{
    "chain_of_thought": "To find Jason's age, we first determine Rick's age. Rick was born 3 years ago, so he is 3 years old. Jason is 25 years older than Rick, so Jason's age is 3 years (Rick's age) + 25 years = 28 years.",
    "name": "Jason",
    "age": 28,
    "email": null
}
